In [2]:
import pandas as pd ## Обновить pandas до последней версии
import numpy as np
import math
import os
import glob
import mysql.connector
import datetime
from mysql.connector import errorcode
from sklearn import preprocessing
from sklearn import metrics

import matplotlib.pyplot as plt
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
from datetime import date
import string

from os import listdir
from os.path import isfile, join


import warnings
warnings.filterwarnings("ignore")

In [3]:
user = 'project_user'
database = 'project_db'

#### Блок Ивахненко Анастасии

In [4]:
my_path = 'C:\\Users\\User\\Documents\\Python Scripts\\IT\\Untitled Folder' 

In [5]:
applications = [f for f in listdir(my_path+'/applications') if isfile(join(my_path+'/applications', f))] 
contracts = [f for f in listdir(my_path+'/contracts') if isfile(join(my_path+'/contracts', f))]

In [6]:
df = pd.read_excel(my_path+'/applications/' + applications[0], header = None, usecols = [0, 3, 4, 5, 6, 7, 8, 12, 14]) 
#Читаем первый excel, чтобы получить названия колонок

In [7]:
columns = [df[0][4], df[0][13], df[0][15], df[0][17], df[0][19], df[1][4], df[2][6], df[2][17], df[3][6], df[4][2], df[4][6],
          df[5][2], df[6][2], df[6][4], df[7][8], 'application_date', df[8][2]]
#список с названиями колнок

In [8]:
values = []
for i in applications:
    df = pd.read_excel(my_path+'/applications/' + i, header = None, usecols = [0, 3, 4, 5, 6, 7, 8, 12, 14])
    if df.shape[0] < 21:
        df.loc[20] = np.nan
    values.append((df[0][5], df[0][14], df[0][16], df[0][18], df[0][20], df[1][5], df[2][7], df[2][18], 
                   df[3][7], df[4][3], df[4][7], df[5][3], df[6][3], df[6][5], df[7][9], df[8][1], df[8][3]))

In [9]:
applic_df = pd.DataFrame(data = values, columns = columns)
applic_df.head(4)

,Identity Number,Income,Income Type,Housing,Age of Car (if owned),Issue Date,Children,House ownership,Family,Date of Birth,Marital Status,Gender,Employed By,Education,Position,application_date,Date appointed
0,100003,270000.0,State servant,House / apartment,NaN,08.29.2013,0.0,N,2.0,07.22.1968,Married,Female,School,Higher education,Core staff,06.16.2014,03.16.2011
1,100004,67500.0,Working,House / apartment,26.0,01.28.2010,0.0,Y,1.0,1964-10-11 00:00:00,Single / not married,Male,Government,Secondary / secondary special,Laborers,2017-02-01 00:00:00,05.22.2016
2,100006,135000.0,Working,House / apartment,NaN,2010-08-01 00:00:00,0.0,Y,2.0,08.29.1964,Civil marriage,Female,Business Entity Type 3,Secondary / secondary special,Laborers,2016-10-09 00:00:00,05.16.2008
3,100007,121500.0,Working,House / apartment,NaN,03.18.2008,0.0,Y,1.0,1963-09-02 00:00:00,Single / not married,Male,Religion,Secondary / secondary special,Core staff,2017-05-09 00:00:00,2009-12-05 00:00:00


In [10]:
applic_df = applic_df.drop(['Date appointed'], axis = 1)
applic_df = applic_df.drop(['Issue Date'], axis = 1)

In [11]:
def calculate_age(born):
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [12]:
ages = []
for i, value in enumerate(applic_df['Date of Birth']):
    try:
        ages.append(calculate_age(datetime.datetime.strptime(applic_df['Date of Birth'][i], '%m.%d.%Y')))
    except:
        ages.append(calculate_age(value))
        
applic_df['age'] = ages

In [13]:
for i, value in enumerate(applic_df['application_date']):
    try:
        applic_df['application_date'].loc[i] = datetime.date.strftime(datetime.datetime.strptime(value, '%m.%d.%Y'), "%Y-%m-%d")
    except:
        applic_df['application_date'].loc[i] = (str(applic_df['application_date'][i].year) + 
                                                '-' + str(applic_df['application_date'][i].month) + '-' + 
                                                str(applic_df['application_date'][i].day))

In [14]:
applic_df = applic_df.where((pd.notnull(applic_df)), None)

In [15]:
applic_df = applic_df.drop(['Date of Birth'], axis = 1)

In [54]:
cnx = mysql.connector.connect(host = 'localhost', database = database, user = user)
cursor = cnx.cursor(buffered=True)

cursor.execute("""CREATE TABLE IF NOT EXISTS applications 
(id INT, 
income INT, 
income_type VARCHAR(30), 
housing VARCHAR (30), 
age_of_car INT,  
children INT,
house_ownership VARCHAR(30), 
family INT,
marital_status VARCHAR(30), 
gender VARCHAR(30),
employed_by VARCHAR(30), 
education VARCHAR(30),
position VARCHAR(30),
application_date DATE,
age INT,
PRIMARY KEY (id)) ENGINE=INNODB""")

In [55]:
# заполняем значениями таблицу applications
sql = "INSERT INTO applications VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
for i in range(0, applic_df.shape[0]):
    val = (int(applic_df.iloc[i]['Identity Number']), applic_df.iloc[i]['Income'], applic_df.iloc[i]['Income Type'],
           applic_df.iloc[i]['Housing'], applic_df.iloc[i]['Age of Car (if owned)'], applic_df.iloc[i]['Children'], 
           applic_df.iloc[i]['House ownership'], applic_df.iloc[i]['Family'], applic_df.iloc[i]['Marital Status'], 
           applic_df.iloc[i]['Gender'], applic_df.iloc[i]['Employed By'], applic_df.iloc[i]['Education'], 
           applic_df.iloc[i]['Position'], applic_df.iloc[i]['application_date'], int(applic_df.iloc[i]['age']))
    cursor.execute(sql, val)

    cnx.commit()

## Contracts

In [18]:
df_contract = pd.read_excel(my_path + '/contracts/' + contracts[0], header = None, usecols = [0, 4, 5])

In [19]:
column_values_contracts = [df_contract[0][4], df_contract[0][6], df_contract[0][8], 
                           df_contract[1][4], df_contract[1][6], df_contract[1][8], 'contract_date']

In [20]:
values_contracts = []
for i in contracts:
    df = pd.read_excel(my_path + '/contracts/' + i, header = None, usecols = [0, 4, 5])
    values_contracts.append((df[0][5], df[0][7], df[0][9], df[1][5], df[1][7], df[1][9], df[2][1]))

In [21]:
contract_df = pd.DataFrame(data = values_contracts, columns = column_values_contracts)
contract_df.head(5)

,Identity Number,Amount,Term (month),Contract Number,Type,Annuity,contract_date
0,100009,1560726.0,41,100022,Cash loans,41301.0,2015-02-11 00:00:00
1,100047,1193580.0,37,100052,Cash loans,35028.0,08.30.2016
2,100025,1132573.5,33,100068,Cash loans,37561.5,12.30.2018
3,100064,298728.0,22,100113,Cash loans,15381.0,02.19.2018
4,100023,544491.0,34,100122,Cash loans,17563.5,07.31.2016


In [22]:
for i, value in enumerate(contract_df['contract_date']):
    try:
        contract_df['contract_date'].loc[i] = datetime.date.strftime(datetime.datetime.strptime(value, '%m.%d.%Y'), "%Y-%m-%d")
    except:
        contract_df['contract_date'].loc[i] = (str(contract_df['contract_date'][i].year) + 
                                                '-' + str(contract_df['contract_date'][i].month) + '-' + 
                                                str(contract_df['contract_date'][i].day))

In [23]:
contract_df = contract_df.where((pd.notnull(contract_df)), None)

In [24]:
cnx.close()

In [56]:
cnx = mysql.connector.connect(host = 'localhost', database = database, user = user)
cursor = cnx.cursor(buffered=True)

cursor.execute("""CREATE TABLE IF NOT EXISTS contracts 
(id INT, 
amount INT, 
term_month INT, 
contract_number INT, 
type VARCHAR(30), 
annuity INT, 
contract_date DATE,
PRIMARY KEY (contract_number),
FOREIGN KEY (id) REFERENCES applications(id)) ENGINE=INNODB """)

In [57]:
sql = "INSERT INTO contracts VALUES (%s, %s, %s, %s, %s, %s, %s)"
for i in range(0, contract_df.shape[0]):
    val = (int(contract_df.iloc[i]['Identity Number']), contract_df.iloc[i]['Amount'], int(contract_df.iloc[i]['Term (month)']),
           int(contract_df.iloc[i]['Contract Number']), contract_df.iloc[i]['Type'], contract_df.iloc[i]['Annuity'], 
           contract_df.iloc[i]['contract_date'])
    try:
        cursor.execute(sql, val)
    except:
        print("Нет заявки с id = %d, такой контракт не передаётся в базу" %contract_df['Identity Number'][i])

    cnx.commit()




Нет заявки с id = 100002, такой контракт не передаётся в базу


## Payments

In [27]:
payments = pd.read_excel(my_path+"/payments.xls")
payments.head(5)

,Contract Number,Date,Amount Due,Amount Paid
0,100875,2016-04-05,24700.5,24700.5
1,100875,2016-05-05,24700.5,24700.5
2,100875,2016-06-05,24700.5,0.0
3,100875,2016-07-05,24700.5,0.0
4,100875,2016-08-05,24700.5,74101.5


In [58]:
cnx = mysql.connector.connect(host = 'localhost', database = database, user = user)
cursor = cnx.cursor(buffered=True)

cursor.execute("""CREATE TABLE IF NOT EXISTS payments 
(contract_number INT, 
payment_date DATE, 
amount_due DECIMAL(18,2), 
amount_paid DECIMAL(18,2),
FOREIGN KEY (contract_number) REFERENCES contracts(contract_number))""")

In [59]:
sql = "INSERT INTO payments VALUES (%s, %s, %s, %s)"
for i in range(0, payments.shape[0]):
    val = (int(payments.iloc[i]['Contract Number']), payments.iloc[i]['Date'], payments.iloc[i]['Amount Due'],
                payments.iloc[i]['Amount Paid'])
    try:
        cursor.execute(sql, val)
    except:
        print("Нет контракта с contract_number = %d, такой платёж не передаётся в базу" %payments['Contract Number'][i])
cnx.commit()

Нет контракта с contract_number = 100875, такой платёж не передаётся в базу
Нет контракта с contract_number = 100875, такой платёж не передаётся в базу
Нет контракта с contract_number = 100875, такой платёж не передаётся в базу
Нет контракта с contract_number = 100875, такой платёж не передаётся в базу
Нет контракта с contract_number = 100875, такой платёж не передаётся в базу
Нет контракта с contract_number = 100875, такой платёж не передаётся в базу
Нет контракта с contract_number = 100875, такой платёж не передаётся в базу
Нет контракта с contract_number = 100875, такой платёж не передаётся в базу
Нет контракта с contract_number = 100875, такой платёж не передаётся в базу
Нет контракта с contract_number = 100875, такой платёж не передаётся в базу
Нет контракта с contract_number = 100875, такой платёж не передаётся в базу
Нет контракта с contract_number = 100875, такой платёж не передаётся в базу
Нет контракта с contract_number = 100875, такой платёж не передаётся в базу


#### Блок Лапшовой Полины

## Кодировка текстовых полей

In [60]:
cnx = mysql.connector.connect(host = 'localhost', database = database, user = user)
cursor = cnx.cursor(buffered=True)

def encoding_and_creating_dictionary(table):
    le = preprocessing.LabelEncoder()
    # создаём таблицу для словаря
    cursor.execute("""CREATE TABLE {0}_dict (column_name VARCHAR(100), category VARCHAR(100), encoding INT)""".format(table))
    # кодируем все текстовые колонки
    cursor.execute("""SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS
    WHERE DATA_TYPE = 'varchar' AND TABLE_SCHEMA = '{0}'
    AND TABLE_NAME = '{1}'""".format(database, table))
    data_type_col = pd.DataFrame(cursor.fetchall())[0].tolist()

    for i in range (0,len(data_type_col)):
        cursor.execute("""SELECT id, {0} FROM {1}""".format(data_type_col[i], table))
        data = pd.DataFrame(cursor.fetchall())
        data.columns = [x[0] for x in cursor.description]
        data = data.dropna(how = 'any').reset_index(drop = True)
        data['encoded'] = le.fit_transform(data[data_type_col[i]])
        cursor.execute("""ALTER TABLE {0} DROP {1}""".format(table, data_type_col[i]))
        cursor.execute("""ALTER TABLE {0} ADD {1} INT""".format(table, data_type_col[i]))
        for j in range (0, len (data)):
            cursor.execute("""UPDATE {0} SET {1} = {2} 
            WHERE {0}.id={3}""".format(table, data_type_col[i], data['encoded'][j], data['id'][j]))
            cnx.commit()
            
            
        # создаём для текущей текстовой колонки таблицу с данными для заполнения словаря
        to_dict = data[[data_type_col[i],'encoded']].drop_duplicates().sort_values(by = ['encoded']).reset_index(drop = True)
        to_dict.columns = ['category','encoding']
        to_dict['column_name'] = data_type_col[i]
        to_dict = to_dict[['column_name', 'category', 'encoding']]

        # заполняем подготовленными данными по текущему текстовому столбцу словарь кодировки
        sql = """INSERT INTO {0}_dict VALUES (%s, %s, %s)""".format(table)
        for k in range(0, to_dict.shape[0]):
            val = (to_dict.iloc[k][0], to_dict.iloc[k][1], int(to_dict.iloc[k][2]))
            cursor.execute(sql, val)

            cnx.commit()

In [61]:
encoding_and_creating_dictionary('applications')

In [62]:
encoding_and_creating_dictionary('contracts')

## Выгрузка данных в форматы csv

In [64]:
cnx = mysql.connector.connect(host = 'localhost', database = database, user = user)
cursor = cnx.cursor(buffered=True)

applications_df = pd.read_sql_query("SELECT * FROM applications", cnx)
contracts_df = pd.read_sql_query("SELECT * FROM contracts", cnx)
applications_dict_df = pd.read_sql_query("SELECT * FROM applications_dict", cnx)
contracts_dict_df = pd.read_sql_query("SELECT * FROM contracts_dict", cnx)

In [78]:
applications_df.to_csv(my_path + '\\output_1\\applications_df.csv', header=True)
contracts_df.to_csv(my_path + '\\output_1\\contracts_df.csv', header=True)
applications_dict_df.to_csv(my_path + '\\output_1\\applications_dict_df.csv', header=True)
contracts_dict_df.to_csv(my_path + '\\output_1\\contracts_dict_df.csv', header=True)